# Speech Intelligibility Index (SII) Example

This notebook demonstrates how to calculate the Speech Intelligibility Index (SII) using wandas.

The SII is a standardized measure of speech intelligibility that ranges from 0.0 (completely unintelligible) to 1.0 (perfectly intelligible), based on ANSI S3.5-1997 standard.

In [ ]:
import numpy as np
import wandas as wd
import matplotlib.pyplot as plt

## Basic Usage

Create a simple speech-like signal and calculate its SII.

In [ ]:
# Generate a speech-like signal with multiple frequency components
signal = wd.generate_sin(
    freqs=[250, 500, 1000, 2000, 4000],
    amplitudes=[0.3, 0.3, 0.2, 0.2, 0.1],
    duration=1.0,
    sampling_rate=44100
)

# Calculate SII
sii_value, specific_sii, freq_axis = signal.sii(method_band="octave")

print(f"Overall SII: {sii_value:.3f}")
print(f"Number of frequency bands: {len(freq_axis)}")

## Visualize Frequency-Specific SII

The SII calculation provides frequency-specific values that show how much each frequency band contributes to overall intelligibility.

In [ ]:
# Plot frequency-specific SII values
plt.figure(figsize=(10, 5))
plt.bar(range(len(specific_sii)), specific_sii, width=0.8)
plt.xticks(range(len(freq_axis)), [f"{f:.0f}" for f in freq_axis], rotation=45)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Specific SII")
plt.title(f"Frequency-Specific SII (Overall SII: {sii_value:.3f})")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Compare Different Method Bands

SII can be calculated using different frequency band methods:
- **octave**: Faster, uses octave bands
- **third octave**: More detailed, uses third-octave bands
- **critical**: Most detailed, uses critical bands

In [ ]:
methods = ["octave", "third octave", "critical"]
results = {}

for method in methods:
    sii_val, spec_sii, freq = signal.sii(method_band=method)
    results[method] = (sii_val, spec_sii, freq)
    print(f"{method:15s}: SII = {sii_val:.3f}, Bands = {len(freq)}")

## Effect of Filtering on SII

Let's see how filtering affects speech intelligibility.

In [ ]:
# Original signal SII
sii_original, _, _ = signal.sii()

# Low-pass filtered signal (preserves low frequencies)
signal_lowpass = signal.low_pass_filter(cutoff=4000)
sii_lowpass, _, _ = signal_lowpass.sii()

# High-pass filtered signal (removes low frequencies)
signal_highpass = signal.high_pass_filter(cutoff=2000)
sii_highpass, _, _ = signal_highpass.sii()

# Band-pass filtered signal
signal_bandpass = signal.band_pass_filter(low_cutoff=300, high_cutoff=3000)
sii_bandpass, _, _ = signal_bandpass.sii()

print("Effect of filtering on SII:")
print(f"Original:          {sii_original:.3f}")
print(f"Low-pass (4kHz):   {sii_lowpass:.3f}")
print(f"High-pass (2kHz):  {sii_highpass:.3f}")
print(f"Band-pass (0.3-3kHz): {sii_bandpass:.3f}")

## Effect of Noise on SII

Add noise to the signal and observe how it affects intelligibility.

In [ ]:
# Clean signal
clean_signal = wd.generate_sin(
    freqs=[500, 1000, 2000],
    amplitudes=[0.3, 0.3, 0.2],
    duration=1.0,
    sampling_rate=44100
)

sii_clean, _, _ = clean_signal.sii()
print(f"Clean signal SII: {sii_clean:.3f}")

# Add different levels of noise
noise_levels = [0.05, 0.1, 0.2, 0.3]
sii_noisy = []

for noise_level in noise_levels:
    # Create noisy signal
    data = clean_signal._data.compute()
    noise = np.random.normal(0, noise_level, data.shape)
    noisy_data = data + noise
    
    noisy_signal = wd.ChannelFrame(
        data=noisy_data,
        sampling_rate=44100
    )
    
    sii_val, _, _ = noisy_signal.sii()
    sii_noisy.append(sii_val)
    print(f"Noise level {noise_level:.2f}: SII = {sii_val:.3f}")

# Plot the effect of noise
plt.figure(figsize=(8, 5))
plt.plot([0] + noise_levels, [sii_clean] + sii_noisy, 'o-', linewidth=2, markersize=8)
plt.xlabel("Noise Level (std)")
plt.ylabel("SII")
plt.title("Effect of Noise on Speech Intelligibility Index")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Working with Real Audio Files

You can calculate SII for real audio files as well.

In [ ]:
# Example with a WAV file (uncomment if you have an audio file)
# audio = wd.read_wav("path/to/your/speech_audio.wav")
# sii_value, specific_sii, freq_axis = audio.sii(method_band="octave")
# print(f"Audio file SII: {sii_value:.3f}")

## References

- ANSI S3.5-1997: Methods for Calculation of the Speech Intelligibility Index
- MoSQITo library: https://mosqito.readthedocs.io/
- Wandas documentation: https://kasahart.github.io/wandas/